In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy.spatial import cKDTree

In [2]:
boro_dfs = ['stat_df.csv', 'queens_df.csv', 'manhatt_df.csv', 'bronx_df.csv', 'bklyn_df.csv']
boros = ['staten-island', 'queens', 'manhattan', 'bronx', 'brooklyn']


In [3]:
lines_agg = '../SimplifiedGeoJson/sublines_aggregate.json'
stations = '../SimplifiedGeoJson/reduced_stations.json'
entrances = '../SimplifiedGeoJson/reduced_subentr.json'

In [4]:
with open(lines_agg, 'r') as fp:
    lines_data = json.load(fp)

with open(stations, 'r') as fp:
    stations_data = json.load(fp)

with open(entrances, 'r') as fp:
    entrance_data = json.load(fp)

In [5]:
# create lines cdk
lines_cdk = {}
for line in lines_data:
    lines_cdk[line] = cKDTree(lines_data[line])


In [7]:
agg_entrances = []
for entr in entrance_data['features']:
    agg_entrances.append(entr['coords'])

In [9]:
entrances_ckd = cKDTree(agg_entrances)

In [10]:
# stations ckd
stations_agg = []
for s in stations_data['features']:
    stations_agg.append(s['coords'])

In [12]:
stations_ckd = cKDTree(stations_agg)

### Add distance to nearest line

In [13]:
def add_dist_to_line(df_path):
    cbdf = pd.DataFrame(pd.read_csv(df_path))
    cbdf['Dist_To_Line'] = None

    for idx, row in cbdf.iterrows():
        lon, lat, line_name = row['Long'], row['Lat'], row['Train_Line']
        dist,_ = lines_cdk[line_name].query([lon, lat], k=1)
        cbdf.at[idx, 'Dist_To_Line'] = dist
    
    cbdf.to_csv(df_path, index=False)
    return

In [14]:
for df in boro_dfs:
    add_dist_to_line(df)

### Add distance to nearest station

In [15]:
def dist_to_station(df_path):
    cbdf = pd.DataFrame(pd.read_csv(df_path))
    cbdf['Dist_To_Station'] = None

    for idx, row in cbdf.iterrows():
        lon, lat = row['Long'], row['Lat']
        dist,_ = stations_ckd.query([lon, lat], k=1)
        cbdf.at[idx, 'Dist_To_Station'] = dist
    
    cbdf.to_csv(df_path, index=False)
    return

In [16]:
for df in boro_dfs:
    dist_to_station(df)

### Add distance to nearest train entrance

In [17]:
def dist_to_entrance(df_path):
    cbdf = pd.DataFrame(pd.read_csv(df_path))
    cbdf['Dist_To_Entrance'] = None

    for idx, row in cbdf.iterrows():
        lon, lat = row['Long'], row['Lat']
        dist,_ = entrances_ckd.query([lon, lat], k=1)
        cbdf.at[idx, 'Dist_To_Entrance'] = dist
    
    cbdf.to_csv(df_path, index=False)
    return

In [ ]:
for df in boro_dfs:
    dist_to_entrance(df)